### Pong

Run this notebook and see what happens. 

Click the game canvas once it shows up to bring it into focus and then use the arrows and keys A and D to control the pedals.

* pong sound is by [freesound](https://freesound.org/people/NoiseCollector/sounds/4359/).
* Commodore 64 font is by [KreativeKorp](https://www.kreativekorp.com/software/fonts/c64.shtml).

In [1]:
import pyglet
import math
import time
import sys
import os

import PIL.Image

import numpy as np

In [2]:
p0 = os.path.abspath('.')
p1 = os.path.abspath(os.path.join(p0, '..'))

sys.path.insert(0, p1)

In [3]:
import jupylet.color

from jupylet.app import App
from jupylet.label import Label
from jupylet.sprite import Sprite

In [4]:
import pyglet.window.key as key

In [5]:
app = App(mode='jupyter')

In [6]:
window = app.window

In [7]:
WIDTH = app.width
HEIGHT = app.height

In [8]:
background = '#3e32a2'
foreground = '#7c71da'

In [9]:
app.set_window_color(foreground)

In [10]:
a0 = np.ones((32, 32)) * 255
a1 = np.ones((128, 16)) * 255
a2 = np.ones((HEIGHT * 9 // 10, WIDTH * 9 // 10, 3)) * jupylet.color.color2rgb(background)[:3]

In [11]:
ball = Sprite(a0, y=HEIGHT/2, x=WIDTH/2, autocrop=True)

padl = Sprite(a1, y=HEIGHT/2, x=48)
padr = Sprite(a1, y=HEIGHT/2, x=WIDTH-48)

field = Sprite(a2, y=HEIGHT/2, x=WIDTH/2) 

In [12]:
sound = pyglet.media.load('sounds/pong-blip.wav', streaming=False)

In [13]:
pyglet.font.add_file('fonts/PetMe64.ttf')

In [14]:
scorel = Label(
    '0', font_name='Pet Me 64', font_size=42, color=foreground, 
    x=64, y=HEIGHT/2, anchor_y='center', anchor_x='left'
)

scorer = Label(
    '0', font_name='Pet Me 64', font_size=42, color=foreground, 
    x=WIDTH-64, y=HEIGHT/2, anchor_y='center', anchor_x='right'
)

In [15]:
sl = 0
sr = 0

In [16]:
@app.event
def on_draw():
    
    window.clear()
    
    field.draw()
    
    scorel.draw()
    scorer.draw()
    
    ball.draw()
    padl.draw()
    padr.draw()

In [17]:
vyl = 0
pyl = HEIGHT/2

vyr = 0
pyr = HEIGHT/2

left = False
right = False

key_a = False
key_d = False

In [18]:
@app.event
def on_key_press(symbol, modifiers):
    
    global left, right, key_a, key_d
    
    if symbol == key.LEFT:
        left = True
        
    if symbol == key.RIGHT:
        right = True
        
    if symbol == key.A:
        key_a = True
        
    if symbol == key.D:
        key_d = True
        

@app.event
def on_key_release(symbol, modifiers):
    
    global left, right, key_a, key_d
    
    if symbol == key.LEFT:
        left = False
        
    if symbol == key.RIGHT:
        right = False

    if symbol == key.A:
        key_a = False
        
    if symbol == key.D:
        key_d = False
        

@app.run_me_again_and_again(1/120)
def update_pads(dt):
    
    global vyl, vyr, pyl, pyr
    
    if right:
        pyr = min(HEIGHT, pyr + dt * 512)
        
    if left:
        pyr = max(0, pyr - dt * 512)
        
    if key_a:
        pyl = min(HEIGHT, pyl + dt * 512)
        
    if key_d:
        pyl = max(0, pyl - dt * 512)
        
    ayl = 200 * (pyl - padl.y)
    vyl = vyl * 0.9 + (ayl * dt)
    
    ayr = 200 * (pyr - padr.y)
    vyr = vyr * 0.9 + (ayr * dt)
    
    padl.y += vyl * dt
    padr.y += vyr * dt
    
    padr.clip_position(WIDTH, HEIGHT)
    padl.clip_position(WIDTH, HEIGHT)

In [19]:
bvx = 192
bvy = 192

In [20]:
@app.run_me_again_and_again(1/120)
def update_ball(dt):
    
    global bvx, bvy, sl, sr

    bs0 = bvx ** 2 + bvy ** 2
    
    ball.rotation += 200 * dt
    
    ball.x += bvx * dt
    ball.y += bvy * dt
    
    if ball.top >= HEIGHT:
        app.play_once(sound)
        ball.y -= ball.top - HEIGHT
        bvy = -bvy
        
    if ball.bottom <= 0:
        app.play_once(sound)
        ball.y -= ball.bottom
        bvy = -bvy
        
    if ball.right >= WIDTH:
        app.play_once(sound)
        ball.x -= ball.right - WIDTH
        
        bvx = -192
        bvy = 192 * np.sign(bvy)
        bs0 = 0
        
        sl += 1
        scorel.text = str(sl)
        
    if ball.left <= 0:
        app.play_once(sound)
        ball.x -= ball.left
        
        bvx = 192
        bvy = 192 * np.sign(bvy)
        bs0 = 0
        
        sr += 1
        scorer.text = str(sr)
        
    if bvx > 0 and ball.top >= padr.bottom and padr.top >= ball.bottom: 
        if 0 < ball.right - padr.left < 10:
            app.play_once(sound)
            ball.x -= ball.right - padr.left
            bvx = -bvx
            bvy += vyr / 2
            
    if bvx < 0 and ball.top >= padl.bottom and padl.top >= ball.bottom: 
        if 0 < padl.right - ball.left < 10:
            app.play_once(sound)
            ball.x += ball.left - padl.right
            bvx = -bvx
            bvy += vyl / 2
            
    bs1 = bvx ** 2 + bvy ** 2
    
    if bs1 < 0.9 * bs0:
        bvx = (bs0 - bvy ** 2) ** 0.5 * np.sign(bvx)

    ball.wrap_position(WIDTH, HEIGHT)

In [21]:
@app.run_me_now()
def highlights(dt):
    
    sl0 = sl
    sr0 = sr
    
    slc = np.array(scorel.color)
    src = np.array(scorer.color)
    
    while True:
        
        dt = yield 1/30
        
        r0 = 0.9 ** (120 * dt)
        
        scorel.color = np.array(scorel.color) * r0 + (1 - r0) * slc
        scorer.color = np.array(scorer.color) * r0 + (1 - r0) * src
        
        if sl0 != sl:
            sl0 = sl
            scorel.color = 'white'

        if sr0 != sr:
            sr0 = sr
            scorer.color = 'white'
            

In [22]:
app.run()

Canvas(layout=Layout(height='512px', width='512px'), size=(512, 512))

In [ ]:
app.stop()